<a href="https://colab.research.google.com/github/TheDon57/MODS_Exercises/blob/main/Week_11_exercise/Topic_Modeling_Week_11_yelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bonuspunktaufgabe 3

In [73]:
!pip install gensim
!pip install nltk
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import nltk
import re
from sklearn.tree import plot_tree
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# Set seed for reproducibility
np.random.seed(42)  # Set seed for NumPy
random.seed(42) # Set seed for random module

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Nutzen Sie dieses Notebook, um die Bonuspunktaufgabe 3 zu bearbeiten. Zuvor werden alle nötigen Packages importiert, die Sie zur Bearbeitung der Aufgabe benötigen.

Ziel ist es, die Yelp-Rezensionen mittels Topic Modeling zu analysieren und Topics aus den Daten zu extrahieren. Anschließend sollen diese Informationen genutzt werden, um mit supervised Machine Learning die Bewertung (`stars`) vorherzusagen.

Nutzen Sie die untenstehende Codezelle, um diese Aufgabe zu bearbeiten.

In [74]:
# Load the data
reviews = pd.read_csv("https://raw.githubusercontent.com/kbrennig/MODS_WS25_26/refs/heads/main/data/yelp.csv")

print(reviews[0:5])

                       id  stars                 date  \
0  ----X0BIDP9tA49U3RvdSQ      4  2014-02-17 00:00:00   
1  ---0hl58W-sjVTKi5LghGw      4  2016-07-24 00:00:00   
2  ---3OXpexMp0oAg77xWfYA      5  2012-04-07 00:00:00   
3  ---65iIIGzHj96QnOh89EQ      5  2015-09-11 00:00:00   
4  ---7WhU-FtzSUOje87Y4uw      5  2016-01-22 00:00:00   

                                                text             business_id  \
0  Red, white and bleu salad was super yum and a ...  Ue6-WhXvI-_1xUIuapl0zQ   
1  Ate the momos during the momo crawl.. Was the ...  Ae4ABFarGMaI5lk1i98A0w   
2  Pizza here made my night... Good people and gr...  lKq4Qsz13FDcAVgp49uukQ   
3  Great brisket sandwich as claimed. Weird that ...  6nKR80xEGHYf2UxAe_Cu_g   
4  Interesting food, great atmosphere, and great ...  Z_mJYg3vi8cPZHa1J4BALw   

                  user_id  
0  gVmUR8rqUFdbSeZbsg6z_w  
1  Y6qylbHq8QJmaCRSlKdIog  
2  SnXZkRN9Yf060pNTk1HMDg  
3  VcmSgvslHAhqWoEn16wjjw  
4  NKF9v-r0jd1p0JVi9h2T1w  


In [75]:
# Split the data
X = reviews.drop(columns=['id','user_id','business_id', 'date'])
y = reviews['stars']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [76]:
# Preprocess the text
def preprocess(text):
  # Tokenizing
  tokens = nltk.word_tokenize(text)

  # Stemming
  stemmer = nltk.stem.PorterStemmer()
  stemmed_tokens = [stemmer.stem(token) for token in tokens]

  # Stopword removal
  stopwords = nltk.corpus.stopwords.words("english")
  filtered_tokens = [token for token in stemmed_tokens if token.lower() not in stopwords]

  # Punctuation removal
  filtered_tokens_nopunct = [re.sub(r'[^\w\s]', '', token) for token in filtered_tokens if token]

  return filtered_tokens_nopunct

# Apply preprocessing
X_train['tokens'] = X_train['text'].apply(preprocess)
X_test['tokens'] = X_test['text'].apply(preprocess)

In [77]:
# Create the Dictionary and Corpus
dictionary = corpora.Dictionary(X_train['tokens'])
dictionary.filter_extremes(no_below=5)

corpus_train = [dictionary.doc2bow(text) for text in X_train['tokens']]
corpus_test = [dictionary.doc2bow(text) for text in X_test['tokens']]

In [78]:
# Train Topic Model
model_10 = LdaModel(corpus=corpus_train, num_topics=10, id2word = dictionary, iterations=10, random_state=42)

In [79]:
# Extract Topic Distributions and Store it in Data Frames
def get_document_topic_distribution(model, corpus):
  return pd.DataFrame(
      [[prob for _, prob in model.get_document_topics(doc, minimum_probability=0)] for doc in corpus],
      columns=[f'topic{i+1}' for i in range(model.num_topics)]
  )

train_topic_distributions = get_document_topic_distribution(model_10, corpus_train)
test_topic_distributions = get_document_topic_distribution(model_10, corpus_test)

In [80]:
# Train and Fit a Decision Tree
decision_tree = DecisionTreeRegressor(random_state=42).fit(train_topic_distributions, y_train)

In [81]:
# Evaluate a Decision Tree
predictions_tree = decision_tree.predict(test_topic_distributions)

rmse_tree = mean_squared_error(y_test, predictions_tree)
print(rmse_tree)

3.0805


In [82]:
# Train and Fit a Linear Regression
train_topic_distributions_ols = train_topic_distributions
train_topic_distributions_ols['stars'] = y_train.values
test_topic_distributions_ols = test_topic_distributions
test_topic_distributions_ols['stars'] = y_test.values

linear_model = ols(
    formula="stars ~ topic1 + topic2 + topic3 + topic4 + topic5 + topic6 + topic7 + topic8 + topic9 + topic10",
    data=train_topic_distributions_ols
)
linear_model = linear_model.fit()

In [83]:
# Evaluate a Linear Regression
predictions_linear = linear_model.predict(test_topic_distributions_ols)

rmse_linear = mean_squared_error(y_test, predictions_linear)
print(rmse_linear)

1.6396564618982181
